In [1]:
from pyspark import SparkContext
from pyspark import SparkConf
from datetime import datetime
from pyspark.sql import SQLContext
from itertools import chain
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.types import Row

In [3]:
ratings = sc.textFile("u.data").map(lambda line: line.split("\t"))
movies = sc.textFile("u.item").map(lambda line: line.split("|"))
users = sc.textFile("u.user").map(lambda line: line.split("|"))

In [ ]:
ratings_data = sc.textFile("u.data")
movies_data = sc.textFile("u.item")
users_data = sc.textFile("u.user")

In [ ]:
r_Schema= StructType([StructField("user id", StringType(), True),
                            StructField("item id", StringType(), True),
                            StructField("rating", StringType(), True),
                            StructField("timestamp", StringType(), True)])
m_Schema= StructType([StructField("movie id", StringType(), True),
                           StructField("movie title", StringType(), True),
                           StructField("release date", StringType(), True),
                           StructField("video release date", StringType(), True),
                           StructField("IMBd URL", StringType(), True),
                           StructField("unknown", StringType(), True),
                           StructField("action", StringType(), True),
                           StructField("adventure", StringType(), True),
                           StructField("animation", StringType(), True),
                           StructField("Childen's", StringType(), True),
                           StructField("comedy", StringType(), True),
                           StructField("crime", StringType(), True),
                           StructField("documentary", StringType(), True),
                           StructField("drama", StringType(), True),
                           StructField("fantasy", StringType(), True),
                           StructField("film-noir", StringType(), True),
                           StructField("horror", StringType(), True),
                           StructField("musical", StringType(), True),
                           StructField("mystery", StringType(), True),
                           StructField("romance", StringType(), True),
                           StructField("sci-fi", StringType(), True),
                           StructField("thriller", StringType(), True),
                           StructField("war", StringType(), True),
                           StructField("western", StringType(), True)])
u_Schema= StructType([StructField("user id", StringType(), True),
                          StructField("age", StringType(), True),
                          StructField("gender", StringType(), True),
                          StructField("occupation", StringType(), True),
                          StructField("zip code", StringType(), True)])

In [ ]:
df_ratings = spark.createDataFrame(ratings, r_Schema).alias("df_ratings")
df_movies = spark.createDataFrame(movies, m_Schema)
df_users = spark.createDataFrame(users, u_Schema)

In [3]:

#This function returns a movie names dict.
def loadMovieNames():
    movieNames = {}
    with open("u.item",encoding = "ISO-8859-1") as f:
        for line in f: # for each line
            fields = line.split('|') #turn line into list of items
            movieNames[int(fields[0])] = fields[1] #adds name to dict with movie id as the name's key.
    return movieNames
def parse_movie_genres(line):
    row = line.split("|")
    return Row(movie_id=int(row[0]), movie_title=row[1], uknown=int(row[5]), action=int(row[6]), adventure=int(row[7]), animation=int(row[8]), children=int(row[9]), comedy=int(row[10]), crime=int(row[11]), documentary=int(row[12]), fantasy=int(row[13]), film_noir=int(row[14]), horror=int(row[15]), musical=int(row[16]), mystery=int(row[17]), romance=int(row[18]), sci_fi=int(row[19]), thriller=int(row[20]), war=int(row[21]), western=int(row[22]))


In [4]:
# def parseInput(line):
#     fields = line.split()
#     return (int(fields[1]), (float(fields[2]), 1.0))


In [4]:
# if __name__ == "__main__":
#     # The main script - create our SparkContext
#     conf = SparkConf().setAppName("WorstMovies")
#     sc = SparkContext.getOrCreate();

# Top 10 Movies

movieNames = loadMovieNames()
df = df_ratings.withColumn("counter", lit(1))
df2 = df.withColumn("user id",df["user id"].cast(IntegerType()))
df3 = df2.groupBy("user id").sum("counter")
df4 = df3.sort("sum(counter)", ascending=False).take(10)
for result in df4:
    print(movieNames[result[0]], result[1])

# Rating by Genre

In [ ]:
df = movies_data.map(parse_movie_genres)
df2 = spark.createDataFrame(df).alias("movies_genres")
df3 = df2.join(df_ratings, df2.movie_id == df_ratings["item id"])
df3.groupBy().sum().take(10)

# Oldest movie with 5 rating

In [ ]:
df = df_movies[["movie id", "movie title", "release date"]]
df2 = df.withColumn("release date",to_date(df["release date"], "dd-MMM-yyyy"))
rating = df_ratings[["item id", "rating"]].filter(col("rating") == 5)
df3 = df2.join(rating, rating["item id"] == df3["movie id"])
df4 = df3.orderBy("release date", ascending=True).where(df3["release date"].isNotNull()).take(1)
df4

# Genre with top 10 movies

In [ ]:
df = df_ratings.withColumn("counter", lit(1))
df2 = df.withColumn("user id",df["user id"].cast(IntegerType()))
df3 = df2.groupBy("user id").sum("counter")
df4 = df3.sort("sum(counter)", ascending=False).take(10)

In [ ]:
df4